# <font color="green"> Problem 1 :  fork, exec, waitの練習</font>
以下を行うプログラムを書け

1. 時刻をナノ秒単位で取得($t_0$とする)
2. 以下を多数回($n$回)繰り返す
 * 子プロセスをforkする
  * 子プロセスはただちに ./do_nothing という, 何もしないプログラムをexecする
  * 親プロセスはただちに子プロセスの終了をwaitする
3. 時刻をナノ秒単位で取得($t_1$とする)
4. 1回あたりの時間($(t_1 - t_0)/n$) をナノ秒単位で出力

* do_nothingは以下のような, 何もしないプログラム

In [ ]:
%%writefile do_nothing.c
int main() {
  return 0;
}

In [ ]:
gcc -Wall do_nothing.c -o do_nothing

* $n$はコマンドラインから取得できるようにする
* 以下のコードを修正して上記を達成せよ

In [ ]:
%%writefile time_fork_exec_wait_ans.c
#include <err.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <time.h>
#include <sys/types.h>
#include <sys/wait.h>

long cur_time() {
  struct timespec ts[1];
  clock_gettime(CLOCK_REALTIME, ts);
  return ts->tv_sec * 1000000000L + ts->tv_nsec;
}

int main(int argc, char ** argv) {
  int n = (argc > 1 ? atoi(argv[1]) : 5);
  long t0 = cur_time();
  for (int i = 0; i < n; i++) {
    pid_t pid = fork();           /* 現プロセスをコピー */
    if (pid == -1) {
      err(1, "fork");
    } else if (pid == 0) {        /* 新しいプロセス(子プロセス) */
      char * const argv[] = { "./do_nothing", 0 };
      execvp(argv[0], argv);
      err(1, "execve");
    } else {
      int ws;
      pid_t cid = waitpid(pid, &ws, 0);
      if (cid == -1) err(1, "waitpid");
    }
  }
  long t1 = cur_time();
  long dt = t1 - t0;
  printf("%ld nsec to fork-exec-wait %d processes (%ld nsec/proc)\n",
         dt, n, dt / n);
  return 0;
}

In [ ]:
gcc -Wall time_fork_exec_wait_ans.c -o time_fork_exec_wait_ans

* 以下のコマンドラインを色々変更して, 1回あたりの時間を計測せよ
* これは概ね, fork + exec + exit + wait の時間 (プログラムを起動して終了するまでにかかる最小の時間)を計測していることに相当する
* 正しく動いているかを確認するために, do_nothingで何かをprintするとか, time_fork_exec_wait中でwaitpidの結果を表示するようにせよ
* 時間を計測するときはそれらの表示を消すこと(消さないと, 測っているのは出力時間が大半を占めることになる)

In [ ]:
./time_fork_exec_wait_ans 10

* 子プロセスが do_nothing を exec する代わりに, 直ちにexit した場合の時間(fork + wait の時間)も計測せよ

In [ ]:
%%writefile time_fork_exit_wait_ans.c
#include <err.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <time.h>
#include <sys/types.h>
#include <sys/wait.h>

long cur_time() {
  struct timespec ts[1];
  clock_gettime(CLOCK_REALTIME, ts);
  return ts->tv_sec * 1000000000L + ts->tv_nsec;
}

int main(int argc, char ** argv) {
  int n = (argc > 1 ? atoi(argv[1]) : 5);
  long t0 = cur_time();
  for (int i = 0; i < n; i++) {
    pid_t pid = fork();           /* 現プロセスをコピー */
    if (pid == -1) {
      err(1, "fork");
    } else if (pid == 0) {        /* 新しいプロセス(子プロセス) */
      exit(0);
    } else {
      int ws;
      pid_t cid = waitpid(pid, &ws, 0);
      if (cid == -1) err(1, "waitpid");
    }
  }
  long t1 = cur_time();
  long dt = t1 - t0;
  printf("%ld nsec to fork-wait %d processes (%ld nsec/proc)\n",
         dt, n, dt / n);
  return 0;
}

In [ ]:
gcc -Wall time_fork_exit_wait_ans.c -o time_fork_exit_wait_ans

In [ ]:
./time_fork_exit_wait_ans 10